In [1]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments,TextDataset, DataCollatorForLanguageModeling
from torch.utils.data import random_split

In [2]:
# Load the CSV file into a Pandas dataframe
df = pd.read_csv('careers_single.csv')

In [3]:
# Initialize the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

/Users/stefanrodrigues/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
# Tokenize the dataset and convert it to a PyTorch dataset
text_dataset = TextDataset(tokenizer=tokenizer, file_path="careers_single.csv", block_size=128)

/Users/stefanrodrigues/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
# Split the dataset into train and validation sets
train_size = int(0.9 * len(text_dataset))
val_size = len(text_dataset) - train_size
train_dataset, val_dataset = random_split(text_dataset, [train_size, val_size])

In [6]:
# Define the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False, 
    pad_to_multiple_of=8
)

In [7]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./", 
    overwrite_output_dir=True, 
    num_train_epochs=3, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    save_total_limit=1,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_steps=50
)

In [8]:
# Define the trainer
trainer = Trainer(
    model=model, 
    args=training_args, 
    data_collator=data_collator, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset
)

In [9]:
# Train the model
trainer.train()

/Users/stefanrodrigues/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8048
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1509
  Number of trainable parameters = 222903552


  0%|          | 0/1509 [00:00<?, ?it/s]

{'loss': 2.9148, 'learning_rate': 1.9867461895294898e-05, 'epoch': 0.02}
{'loss': 1.9312, 'learning_rate': 1.9734923790589798e-05, 'epoch': 0.04}
{'loss': 0.753, 'learning_rate': 1.960238568588469e-05, 'epoch': 0.06}
{'loss': 0.247, 'learning_rate': 1.946984758117959e-05, 'epoch': 0.08}
{'loss': 0.1668, 'learning_rate': 1.9337309476474488e-05, 'epoch': 0.1}
{'loss': 0.1206, 'learning_rate': 1.9204771371769388e-05, 'epoch': 0.12}
{'loss': 0.0917, 'learning_rate': 1.907223326706428e-05, 'epoch': 0.14}
{'loss': 0.0746, 'learning_rate': 1.893969516235918e-05, 'epoch': 0.16}
{'loss': 0.0536, 'learning_rate': 1.8807157057654078e-05, 'epoch': 0.18}
{'loss': 0.0419, 'learning_rate': 1.8674618952948974e-05, 'epoch': 0.2}
{'loss': 0.0399, 'learning_rate': 1.854208084824387e-05, 'epoch': 0.22}
{'loss': 0.0412, 'learning_rate': 1.8409542743538767e-05, 'epoch': 0.24}
{'loss': 0.0352, 'learning_rate': 1.8277004638833667e-05, 'epoch': 0.26}
{'loss': 0.0311, 'learning_rate': 1.8144466534128564e-05, 'e

Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Configuration saved in ./checkpoint-500/generation_config.json


{'loss': 0.0077, 'learning_rate': 1.3373094764744865e-05, 'epoch': 0.99}


...

Model weights saved in ./checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 895
  Batch size = 16


  0%|          | 0/56 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.0013568727299571037, 'eval_runtime': 91.8732, 'eval_samples_per_second': 9.742, 'eval_steps_per_second': 0.61, 'epoch': 3.0}
{'train_runtime': 14997.2764, 'train_samples_per_second': 1.61, 'train_steps_per_second': 0.101, 'train_loss': 0.051731910956486556, 'epoch': 3.0}


TrainOutput(global_step=1509, training_loss=0.051731910956486556, metrics={'train_runtime': 14997.2764, 'train_samples_per_second': 1.61, 'train_steps_per_second': 0.101, 'train_loss': 0.051731910956486556, 'epoch': 3.0})

In [10]:
# Save the fine-tuned model
model.save_pretrained('fine-tuned-t5-base')
tokenizer.save_pretrained('fine-tuned-t5-base')

Configuration saved in fine-tuned-t5-base/config.json
Configuration saved in fine-tuned-t5-base/generation_config.json
Model weights saved in fine-tuned-t5-base/pytorch_model.bin
tokenizer config file saved in fine-tuned-t5-base/tokenizer_config.json
Special tokens file saved in fine-tuned-t5-base/special_tokens_map.json


('fine-tuned-t5-base/tokenizer_config.json',
 'fine-tuned-t5-base/special_tokens_map.json',
 'fine-tuned-t5-base/spiece.model',
 'fine-tuned-t5-base/added_tokens.json')

In [2]:
# Load the pre-trained and fine-tuned T5 models
pt_model_name = "t5-base"
pt_tokenizer = T5Tokenizer.from_pretrained(pt_model_name)
pt_model = T5ForConditionalGeneration.from_pretrained(pt_model_name)
ft_model_name = "fine-tuned-t5-base"
ft_tokenizer = T5Tokenizer.from_pretrained(ft_model_name)
ft_model = T5ForConditionalGeneration.from_pretrained(ft_model_name)

/Users/stefanrodrigues/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [62]:
# Define the input prompt
prompt = "My dad applied for the position of kindergarten teacher."

In [3]:
#Generate Output
def generate_output(prompt, model, tokenizer):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # Generate text completion
    output_ids = model.generate(input_ids, max_length=100, do_sample=True, temperature=1.5, top_k=50,repetition_penalty=2.0, top_p=0.95)
    # Decode the generated tokens back into text
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

In [7]:
# prompt = "My dad applied for the position of kindergarten teacher."
prompts = [
    "My dad applied for the position of kindergarten teacher."
]
for prompt in prompts:
    print("Prompt:", prompt)
    pt_output = generate_output(prompt, pt_model, pt_tokenizer)
    print("Pre-trained T5 output:", pt_output)
    ft_output = generate_output(prompt, ft_model, ft_tokenizer)
    print("Fine-tuned T5 output:", ft_output)

Prompt: My dad applied for the position of kindergarten teacher.
Pre-trained T5 output: On December 25, I turned 14. Many thanks for your post.
Fine-tuned T5 output: (My dad applied for the position of kindergarten teacher.) My dad applied from an educational background in England during his first school year to become a teacher.
